## Setup (must run cells marked *)


`pip install beautifulsoup4 bertopic flair keybert keyphrase_vectorizers libpff-python scikit-learn`

#### Imports*


In [ ]:
import os
import json
import random
import textwrap

import pandas as pd

#### Get emails from PST file(s)

In [4]:
from pst_extract import get_messages_from_pst

# File path or folder with PSTs. 
pst_input = 'data'
output_folder = 'test_psts'

get_messages_from_pst(pst_input, output_folder)

OSError: pypff_file_open: unable to open file. libcfile_file_open_wide_with_error_code: access denied to file: \\?\c:\projects\email_classifier\data.  libcfile_file_open_wide: unable to open file.  libbfio_file_io_handle_open: unable to open file: \\?\c:\projects\email_classifier\data.  libbfio_handle_open: unable to open handle.  libpff_file_open_file_io_handle: unable to open file IO handle.  libpff_file_open_wide: unable to open file: data.

#### Specify folder contaning email text, one email per file*

In [ ]:
email_folder = None

## Keyword Extraction

#### Setup


In [ ]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

kw_model = KeyBERT()

def get_keywords(text, kwargs):
    if kwargs.get('vectorizer'):
        if kwargs['vectorizer'] == 'keyphrase':
            kwargs['vectorizer'] = KeyphraseCountVectorizer()
        else:
            kwargs['vectorizer'] = CountVectorizer(
                ngram_range=kwargs.get('keyphrase_ngram_range', (1, 1)),
                stop_words=kwargs['stop_words']
            )
    keywords = kw_model.extract_keywords(text, **kwargs)
    keywords = [x[0] for x in keywords]
    print('KEYWORDS:\n')
    print('\n'.join(keywords))

#### Extract keywords from a random or specified email (see KWARGS)

In [ ]:
email_path = os.path.join(email_folder, (random.choice(os.listdir(email_folder))))
email = open(email_path, encoding='utf-8').read()

KWARGS = {
    'keyphrase_ngram_range': (1, 3), # Min, max word count for keywords
    'use_mmr': True, # Increases diversity of keywords
    'diversity': .5, # Set diversity between 0 and 1 if using MMR
    'vectorizer': 'keyphrase', # ("keyphrase", True, False) How to represent document. Keyphrase vectorizer should be more coherent
    'stop_words': 'english'
}

print(f'FILENAME: {email_path}\n')
print('TEXT:\n ', "\n".join(textwrap.wrap(email, 100)), '\n')
get_keywords(email, KWARGS)

# Run on same email with different args
# print()
# KWARGS['vectorizer'] = True
# get_keywords(email, KWARGS)

## Topic Modeling

#### Setup

In [ ]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from bertopic.vectorizers import ClassTfidfTransformer

#### Get topics and view topic hierarchy (see comments)

In [ ]:
# Select subset or all emails in email folder
slice_docs = (None, None)
docs = [open(os.path.join(email_folder, filename), encoding='utf-8').read() for 
        filename in os.listdir(email_folder)[slice_docs[0]:slice_docs[1]]]

# Document representations to chain and feed into topic model
representations = [
    KeyBERTInspired(), # Should make topics more coherent
    MaximalMarginalRelevance(diversity=0.3), # Makes topics more diverse
]

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

topic_model = BERTopic(
        representation_model=representations, # Representations (see above)
        ctfidf_model=ctfidf_model,  # Prevents very frequent words in data from being candidate topics
        nr_topics='auto' # Topic reduction. Set number of desired topics, 'auto' for auto-reduction, 
        # or None. Set to None if there aren't enough topic. 
        )
topics, probabilities = topic_model.fit_transform(docs)
hierarchical_topics = topic_model.hierarchical_topics(docs)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

#### View topics as table

In [ ]:
topic_model.get_topic_info()

#### Show docs per topic(s)

In [ ]:
# Select topic(s) by number (must be list). Get topic numbers from table or 
# labels in topic tree , ie. "1_trees_grass_nature" topic number is 1. 
topics_to_show = [0]
# Show first n docs
n_docs_to_show = 10
# Show first n characters of each doc
n_chars_per_doc = 500

df = pd.DataFrame({"Document": docs, "Topic": topics})
df = df.loc[df['Topic'].isin(topics_to_show)].head(n_docs_to_show)
docs_ = df['Document'].to_list()
random.shuffle(docs_)
for i, doc in enumerate(docs_):
    print(f'DOC {i + 1}\n----------\n')
    print('\n'.join(textwrap.wrap(doc[:n_chars_per_doc] + ' ...' if n_chars_per_doc else '')), '\n')

### Get person-organization and sender/recipient Pairs

In [ ]:
from get_pairs import get_keywords, get_entities, get_person_org_pairs,  get_all_sender_recipient_pairs

# SPECIFY THIS
DOCS_FOLDER = 'emails_small'
#
KW_PATH = 'kws.json'
KW_BATCH_SIZE = None
ENTITY_PATH = 'entities.json'
RELEVANCE_PATH = 'relevance.json'
RELEVANCE_LABEL = 'invoice synset only'
REF_SYNSETS = ['invoice.n.01']
ENTITY_PAIRS_PATH = 'person-org pairs.csv'
TO_FROM_PAIRS_PATH = 'to_from_pairs.csv'
RANKED_ENTITIES_PATH = 'ranked_entities.txt'
KW_KWARGS = {'top_n': 10}
GET_KWS = True
GET_ENTITIES = True
GET_PAIRS = True

if GET_KWS:
    get_keywords(KW_PATH,
                  DOCS_FOLDER,
                    KW_KWARGS,
                      batch_size=KW_BATCH_SIZE)

if GET_ENTITIES:
    get_entities(
        ENTITY_PATH,
        DOCS_FOLDER,
        KW_PATH,
        RELEVANCE_PATH,
        RELEVANCE_LABEL,
        relevance_func_args={
            'ref_synsets': REF_SYNSETS
        })

if GET_PAIRS:
    person_org_pairs = get_person_org_pairs(
        RELEVANCE_LABEL,
        DOCS_FOLDER,
        RELEVANCE_PATH,
        ENTITY_PATH,
        KW_PATH,
        RANKED_ENTITIES_PATH
    )
    person_org_pairs.head(5000).to_csv(ENTITY_PAIRS_PATH)
    to_from_pairs = get_all_sender_recipient_pairs(
            RELEVANCE_PATH, DOCS_FOLDER, RELEVANCE_LABEL)
    to_from_pairs.head(5000).to_csv(TO_FROM_PAIRS_PATH)